In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sc
# import sumo as sm
import os
from traffic_initialization import *
import sys
from bs4 import BeautifulSoup
import tqdm
import pandas as pd

# New handy interface is here!
# Check interface.py for possible parameters
from interface import Simulation
%env SUMO_HOME=/usr/share/sumo
from datetime import datetime
%load_ext autotime

env: SUMO_HOME=/usr/share/sumo
time: 52.3 µs (started: 2024-04-24 19:57:14 +03:00)


# Optimization with annealing

In [2]:
# simple throughput function with two gaps parameters

def loss(d0, d1, N):

    ans = 0
    
    for n in range(10):

        sim = Simulation(start_road='E0', end_road='E2', N=N, work_dir='./xml_data/test6', net_data='test6.net.xml')
        sim.set_calibrator(road='E0', gap=3)
        sim.set_calibrator(road='E1', lane=['0', '1'], gap=d0)
        sim.set_calibrator(road='E1', lane=['2'], gap=d1)
        sim.set_calibrator(road='E1', lane=['3'], gap=2)
        sim.set_calibrator(road='E2', gap=1.5)
        sim.run()
        ans += sim.get_mean_throughput()

    return -ans/10

time: 268 µs (started: 2024-04-24 19:57:14 +03:00)


# Optimization with ML

In [3]:
from Some_models import OptimalLoss_Q, OptimalDist

2024-04-24 19:57:14.788130: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-24 19:57:14.788378: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-24 19:57:14.790130: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-24 19:57:14.811696: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-24 19:57:15.201402: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

time: 979 ms (started: 2024-04-24 19:57:14 +03:00)


In [6]:
# complex throughput function with three gaps parameters

def loss(d0, d1, d2, N):

    ans = 0
    
    for n in range(3):

        sim = Simulation(start_road='E0', end_road='E2', N=N, work_dir='./xml_data/test6', net_data='test6.net.xml')
        sim.set_calibrator(road='E0', gap=3)
        sim.set_calibrator(road='E1', lane=['0'], gap=d0)
        sim.set_calibrator(road='E1', lane=['1'], gap=d1)
        sim.set_calibratоor(road='E1', lane=['2'], gap=d2)
        sim.set_calibrator(road='E1', lane=['3'], gap=2)
        sim.set_calibrator(road='E2', gap=1.5)
        sim.run()
        ans += sim.get_mean_throughput()

    return -ans/3

time: 298 µs (started: 2024-04-24 19:57:37 +03:00)


In [11]:
model = OptimalDist(grid_params=[(4000, 8000, 41), (10, 50, 25), (10, 40, 25), (2, 30, 25)])
model.fit_func(loss)

  0%|          | 0/98 [00:33<?, ?it/s]


ValueError: cannot set a row with mismatched columns

time: 33.3 s (started: 2024-04-24 20:08:33 +03:00)


# Creating of csv

In [3]:
throughput(20, 4, 6000)

1.2891872320004407

time: 27.8 s (started: 2024-04-24 18:31:17 +03:00)


In [4]:
throughput(20, 4, 6000)

1.2932371519835026

time: 27.7 s (started: 2024-04-24 18:31:45 +03:00)


In [18]:
cost(30, 20, 10, 6000)

1.2176827071439196

In [19]:
cost(30, 20, 10, 6000)

1.2098426779299611

In [10]:
cost(30, 20, 10, 3)

0.028846153846153848

In [ ]:
while(True):
    # test6
    filename='csv_data/test6.csv'
    csv_data = pd.read_csv(filename)
    
    N = np.random.randint(3000, 8000)
    for i in range(500):
        rand = np.random.rand(3)
        d0 = 10 + 40*rand[0]
        d1 = 10 + 30*rand[1]
        d2 = 3 + 30*rand[2]
        
        csv_data.loc[len(csv_data)] = (d0, d1, d2, N, sim.get_mean_throughput())
        csv_data.to_csv(filename, index=False) 